# Random Forest Weighted Entropy
As metnioned in the previous notebook, there are different methods to handle missing values in a random forest. A problem with a lot of them is that a lot of them train on the missing data in the train set, to classify the missing data in the test set better. However, what I want to achieve is a better classification on a test set without missing data, so most of those algorithms won't work for this.

During one of the thesis meeting sessions, we brainstormed which possible approach would be best. Our guess was weighted branches. However, usually this trains weights during training, and during testing, it discovers all possible branches, and creates a weighted sum of all those probabilities. This is of course not useful if there isn't any missing data in the test set.

So we need to find a way to use this during training. I'll attempt this using a weighted entropy.

This is the current calculate_entropy function (located in  CustomForest/decision_tree.py):

In [1]:
import math
import numpy as np
import pandas as pd


def calculate_entropy(y):
    """ Calculate the entropy of label array y """
    log2 = lambda x: math.log(x) / math.log(2)
    unique_labels = np.unique(y)
    entropy = 0
    for label in unique_labels:
        count = len(y[y == label])
        p = count / len(y)
        print(f"p({label}) = {p}")
        entropy += -p * log2(p)
    return entropy

So for each unique label in our tree node, we calculate the relative frequncy of that label in the node (p) and multiply it with the log2 of that frequency (which is negative, since probabilities are between 0 and 1), but it's made positive again using the minus sign. We then sum all those values together, and return the entropy.

Let's give an example:

In [24]:
y = np.array([1, 1, 1, 0])
calculate_entropy(y)

p(0) = 0.25
p(1) = 0.75


0.8112781244591328

Instead of those labels, we now also have a weight for each of the instances and calculate the weighted entropy. All those weights start with 1, but get halved when a split has occured of which the value for the corresponding data instance is missing.

If the data is random, p is lower. `p = count / len(y)`. We want to decrease the penalty (so increas p) if there are lower weihgts and increase the penality if there are higher weights. WARNING: All without mising data should be better than with missing data in the pure values.

We also want lower values to have less impact on the count.

Some notes when searching for a new entropy function:
![](img/entropy_notes.png)

In [25]:
weights = np.ones(len(y))

weights[0] = 0.5

np.concatenate((y.reshape(-1, 1), weights.reshape(-1, 1)), axis=1)

array([[1. , 0.5],
       [1. , 1. ],
       [1. , 1. ],
       [0. , 1. ]])

In [65]:
def calculate_weighted_entropy(y, weights):
    """ Calculate the weighted entropy of label array y """
    log2 = lambda x: math.log(x) / math.log(2)
    unique_labels = np.unique(y)
    weighted_sum = np.sum(weights)
    most_common_label = unique_labels[np.argmax([len(y[y == label]) for label in unique_labels])]
    entropy = 0
    entropy_text_formula = ""
    for label in unique_labels:
        label_weights = weights[y == label]

        label_count = len(label_weights)

        p = label_count / len(y)

        if label == most_common_label:
            weighted_count = sum(label_weights)
            p_weighted = weighted_count / weighted_sum
            diff = p_weighted - p
            print(f"Diff: {diff}")
            print(f"Using weighted p ({p_weighted}) instead of p ({p})")
            p = p_weighted


        label_weight = np.prod(label_weights)
        entropy += -p * log2(p)
        entropy_text_formula += f"-{p} * log2({p}) (for label {label}) + "
    print(entropy_text_formula)
    return entropy

Let's verify our entropy is the same if all weights are 1, our entropy is lower if the weights of the ones are lowered and higher if the weights of the zeros are lowered (a higher entropy means more random, less grouped).

In [69]:
# ones lowered, should be higher
weights = np.array([0.5, 1, 1, 1])
calculate_weighted_entropy(y, weights)

Diff: -0.0357142857142857
Using weighted p (0.7142857142857143) instead of p (0.75)
-0.25 * log2(0.25) (for label 0) + -0.7142857142857143 * log2(0.7142857142857143) (for label 1) + 


0.8467334479787441

In [70]:
# weights all 1, should be the same
weights = np.array([1, 1, 1, 1])
calculate_weighted_entropy(y, weights)

Diff: 0.0
Using weighted p (0.75) instead of p (0.75)
-0.25 * log2(0.25) (for label 0) + -0.75 * log2(0.75) (for label 1) + 


0.8112781244591328

In [71]:
# zeros lowered, should be lower
weights = np.array([1, 1, 1, 0.5])
calculate_weighted_entropy(y, weights)

Diff: 0.1071428571428571
Using weighted p (0.8571428571428571) instead of p (0.75)
-0.25 * log2(0.25) (for label 0) + -0.8571428571428571 * log2(0.8571428571428571) (for label 1) + 


0.6906220754312411

This seems to work as intended. Let's try another example:

In [73]:
labels = np.array([1, 1, 1, 0, 0, 0, 0, 0])

In [74]:
weights = np.array([1, 1, 1, 1, 1, 1, 1, 1])
calculate_weighted_entropy(labels, weights)

Diff: 0.0
Using weighted p (0.625) instead of p (0.625)
-0.625 * log2(0.625) (for label 0) + -0.375 * log2(0.375) (for label 1) + 


0.9544340029249649

In [75]:
weights = np.array([1, 1, 1, 1, 1, 1, 1, 0.5])
calculate_weighted_entropy(labels, weights)

Diff: -0.025000000000000022
Using weighted p (0.6) instead of p (0.625)
-0.6 * log2(0.6) (for label 0) + -0.375 * log2(0.375) (for label 1) + 


0.9728184187292901

In [76]:
weights = np.array([1, 1, 1, 1, 1, 1, 1, 0.25])
calculate_weighted_entropy(labels, weights)

Diff: -0.03879310344827591
Using weighted p (0.5862068965517241) instead of p (0.625)
-0.5862068965517241 * log2(0.5862068965517241) (for label 0) + -0.375 * log2(0.375) (for label 1) + 


0.9823221179507028

In [77]:
weights = np.array([0.5, 1, 1, 1, 1, 1, 1, 1])
calculate_weighted_entropy(labels, weights)

Diff: 0.04166666666666663
Using weighted p (0.6666666666666666) instead of p (0.625)
-0.6666666666666666 * log2(0.6666666666666666) (for label 0) + -0.375 * log2(0.375) (for label 1) + 


0.9206140627103372

Now we have a new entropy function, we need to implement the weights. In the build_tree function, they should get initialized with 1 (unless it's a subtree, then the value should be passed on). If the split occurs on a missing value, the weights should be halved.

Because the split always happen on the entropy and the test set doesn't contain missing values, we won't change the evaluation function.

In [1]:
from util import evaluate_no_cv
from CustomForest import WeightedClassificationTree
import numpy as np

tree = WeightedClassificationTree()

y = np.array([1, 1, 1, 0])
X = np.array([[1, 1], [1, 2], [1, 3], [2, 1]])

tree.fit(X, y)

C:\Users\arnod\PycharmProjects\thesis\CustomForest\utils.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([X_1, X_2])


Diff: 0.0
Using weighted p (0.75) instead of p (0.75)
-0.25 * log2(0.25) (for label 0) + -0.75 * log2(0.75) (for label 1) + 


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [5]:
from CustomForest import ClassificationTree
tree = ClassificationTree()
tree.fit(X, y)
tree.predict([[2, 1], [1, 1], [1, 3], [2, 1]])

[1, 1, 1, 1]

In [3]:
from util import evaluate_no_cv
from CustomForest import WeightedClassificationTree
import numpy as np

tree = WeightedClassificationTree()

y = np.array([1, 1, 1, 0])
X = np.array([[1, np.nan], [np.nan, 2], [np.nan, 3], [2, 1]])

tree.fit(X, y)
tree.predict([[2, 1], [1, 1], [1, 3], [2, 1]])

Diff: [-0.08333333]
Using weighted p ([0.66666667]) instead of p (0.75)
-0.25 * log2(0.25) (for label 0) + -[0.66666667] * log2([0.66666667]) (for label 1) + 
Diff: 0.0
Using weighted p (0.5) instead of p (0.5)
-0.5 * log2(0.5) (for label 0.0) + -0.5 * log2(0.5) (for label 1.0) + 
Diff: 0.0
Using weighted p (1.0) instead of p (1.0)
-1.0 * log2(1.0) (for label 1.0) + 
Diff: [-0.08333333]
Using weighted p ([0.66666667]) instead of p (0.75)
-0.25 * log2(0.25) (for label 0) + -[0.66666667] * log2([0.66666667]) (for label 1) + 
Diff: 0.0
Using weighted p (1.0) instead of p (1.0)
-1.0 * log2(1.0) (for label 0.0) + 
Diff: 0.0
Using weighted p (1.0) instead of p (1.0)
-1.0 * log2(1.0) (for label 1.0) + 
Diff: [-0.15]
Using weighted p ([0.6]) instead of p (0.75)
-0.25 * log2(0.25) (for label 0) + -[0.6] * log2([0.6]) (for label 1) + 
Diff: -0.16666666666666663
Using weighted p (0.5) instead of p (0.6666666666666666)
-0.3333333333333333 * log2(0.3333333333333333) (for label 0.0) + -0.5 * log2(0.